In [1]:
import pandas as pd

In [2]:
#Read each of the files
antro_1 = pd.read_csv('./Raw/meta_westernization_paper.meta', sep = '\t')
antro_2 = pd.read_csv('./Raw/meta_diet_paper.meta', sep = '\t')
nutrients = pd.read_csv('./Raw/nutrients.txt', sep = '\t')
otu = pd.read_csv('./Raw/otu_data.otus',sep = '\t')
food_group = pd.read_csv('./Raw/food_groups.csv')

*Overview of data files*
1. Antro_1: 446 observations, ID type MI_001_H
2. Antro_2: 446 observations, ID type MI_001_H 
3. nutrients: 459 observations, ID type MI_001_H (column called id)
4. otu: 446 observations, ID type MI_001_H (column called samples)
5. food_group: 557 observations. Has no ID columnn but is connected to nutrients through Codalt

In [3]:
antro_1.head(1)

ID      city   sex  age age_range  HDL LDL  cholesterol  \
0  MI_001_H  Medellin  Male   36     18_40   38  93          170   

   triglycerides hsCRP  ...  waist medicament fiber per_protein  \
0            195  1,71  ...   97,6         No    19       16,15   

  per_animal_protein per_total_fat per_saturated_fat  per_monoinsaturated_fat  \
0              60,53          28,4             11,01                     9,88   

   per_polyunsaturated_fat per_carbohydrates  
0                     4,82             55,47  

[1 rows x 32 columns]

In [4]:
antro_1.shape

(446, 32)

### Step 1: Verify data types 

#### Step 1.1: Antro_1 

In [5]:
#Antro 1: Anthropometric with per_x columns
antro_1.dtypes

ID                          object
city                        object
sex                         object
age                          int64
age_range                   object
HDL                          int64
LDL                         object
cholesterol                  int64
triglycerides                int64
hsCRP                       object
glucose                      int64
glycosylated_hg             object
adiponectin                 object
insulin                     object
HOMA_IR                     object
stool_consistency           object
hiden_blood                 object
systolic_bp                float64
diastolic_bp                 int64
bmi                         object
bmi_class                   object
body_fat                    object
waist                       object
medicament                  object
fiber                        int64
per_protein                 object
per_animal_protein          object
per_total_fat               object
per_saturated_fat   

In [6]:
#Columns with object data type:
tipos_incorrectos_antro_1 = antro_1.columns[antro_1.dtypes == 'object'].tolist()
tipos_incorrectos_antro_1

['ID',
 'city',
 'sex',
 'age_range',
 'LDL',
 'hsCRP',
 'glycosylated_hg',
 'adiponectin',
 'insulin',
 'HOMA_IR',
 'stool_consistency',
 'hiden_blood',
 'bmi',
 'bmi_class',
 'body_fat',
 'waist',
 'medicament',
 'per_protein',
 'per_animal_protein',
 'per_total_fat',
 'per_saturated_fat',
 'per_monoinsaturated_fat',
 'per_polyunsaturated_fat',
 'per_carbohydrates']

In [7]:
#Remove those meant to be strings
tipos_incorrectos_antro_1.remove('ID')
tipos_incorrectos_antro_1.remove('sex')
tipos_incorrectos_antro_1.remove('city')
tipos_incorrectos_antro_1.remove('age_range')
tipos_incorrectos_antro_1.remove('stool_consistency')
tipos_incorrectos_antro_1.remove('hiden_blood')
tipos_incorrectos_antro_1.remove('bmi_class')
tipos_incorrectos_antro_1.remove('medicament')

In [8]:
antro_1[tipos_incorrectos_antro_1].head(2)

LDL hsCRP glycosylated_hg adiponectin insulin   HOMA_IR   bmi     body_fat  \
0   93  1,71            5,47        5,33   15,09  3,201092  29,7  37,55085213   
1  130  0,32            5,81        2,57   13,57  2,912122  30,4  41,00993849   

  waist per_protein per_animal_protein per_total_fat per_saturated_fat  \
0  97,6       16,15              60,53          28,4             11,01   
1  89,3       13,77              66,88         29,74             11,61   

  per_monoinsaturated_fat per_polyunsaturated_fat per_carbohydrates  
0                    9,88                    4,82             55,47  
1                   10,35                    6,55              56,1

In [9]:
#Multiple columns are wrong because of "," as decimal separator -> we convert them
antro_1[tipos_incorrectos_antro_1] = antro_1[tipos_incorrectos_antro_1].apply(lambda x: pd.to_numeric(x.str.replace(',', '.'), errors='coerce'))

In [10]:
#Verify if our corrections were effective
antro_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446 entries, 0 to 445
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       446 non-null    object 
 1   city                     446 non-null    object 
 2   sex                      446 non-null    object 
 3   age                      446 non-null    int64  
 4   age_range                446 non-null    object 
 5   HDL                      446 non-null    int64  
 6   LDL                      444 non-null    float64
 7   cholesterol              446 non-null    int64  
 8   triglycerides            446 non-null    int64  
 9   hsCRP                    446 non-null    float64
 10  glucose                  446 non-null    int64  
 11  glycosylated_hg          446 non-null    float64
 12  adiponectin              446 non-null    float64
 13  insulin                  446 non-null    float64
 14  HOMA_IR                  4

2 Columns have missing data: LDL, SBP. However, we don't want to eliminate those rows because there is much more information in other columns, so we save those IDs

In [11]:
#Only 3 rows have at least 1 NA
rows_with_na = antro_1[antro_1.isna().any(axis=1)]
ids_with_nas = rows_with_na['ID'].tolist()
ids_with_nas

['MI_017_H', 'MI_078_H', 'MI_354_H']

#### Step 1.2: Antro_2

In [12]:
antro_2.dtypes

ID               object
city             object
sex              object
age_range        object
bmi_class        object
socioeconomic     int64
dtype: object

In [13]:
antro_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446 entries, 0 to 445
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             446 non-null    object
 1   city           446 non-null    object
 2   sex            446 non-null    object
 3   age_range      446 non-null    object
 4   bmi_class      446 non-null    object
 5   socioeconomic  446 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 21.0+ KB


All correct

#### Step 1.3: nutrients 

In [14]:
nutrients.dtypes

id                        object
Codalt                     int64
Calories                   int64
Proteins                 float64
Total_fat                  int64
SFA                      float64
MUFA                     float64
PUFA                     float64
Cholesterol                int64
Carbohydrates              int64
Complex_carbohydrates      int64
Fiber                      int64
Ca                         int64
P                          int64
Fe                       float64
Na                       float64
K                          int64
Mg                         int64
Zn                       float64
Cu                       float64
Mn                       float64
VitA                       int64
B1                       float64
B2                       float64
B3                         int64
B5                         int64
B6                       float64
Folate                     int64
B12                      float64
VitC                       int64
dtype: obj

In [15]:
nutrients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459 entries, 0 to 458
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     459 non-null    object 
 1   Codalt                 459 non-null    int64  
 2   Calories               459 non-null    int64  
 3   Proteins               459 non-null    float64
 4   Total_fat              459 non-null    int64  
 5   SFA                    459 non-null    float64
 6   MUFA                   459 non-null    float64
 7   PUFA                   459 non-null    float64
 8   Cholesterol            459 non-null    int64  
 9   Carbohydrates          459 non-null    int64  
 10  Complex_carbohydrates  459 non-null    int64  
 11  Fiber                  459 non-null    int64  
 12  Ca                     459 non-null    int64  
 13  P                      459 non-null    int64  
 14  Fe                     459 non-null    float64
 15  Na    

Todo bien

#### Step 1.4: Food_groups 

In [16]:
food_group.dtypes

Code            float64
Codalt            int64
City             object
r24h              int64
Dairy g         float64
Meats g         float64
Eggs g          float64
Beans g         float64
Nuts g          float64
Fruits g        float64
Vegetables g    float64
Cereals g       float64
Tubers g        float64
Fats g          float64
Sugars g        float64
HEI             float64
SCORE_GABAS     float64
dtype: object

In [17]:
food_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Code          557 non-null    float64
 1   Codalt        557 non-null    int64  
 2   City          557 non-null    object 
 3   r24h          557 non-null    int64  
 4   Dairy g       557 non-null    float64
 5   Meats g       557 non-null    float64
 6   Eggs g        557 non-null    float64
 7   Beans g       557 non-null    float64
 8   Nuts g        557 non-null    float64
 9   Fruits g      557 non-null    float64
 10  Vegetables g  557 non-null    float64
 11  Cereals g     557 non-null    float64
 12  Tubers g      557 non-null    float64
 13  Fats g        557 non-null    float64
 14  Sugars g      557 non-null    float64
 15  HEI           556 non-null    float64
 16  SCORE_GABAS   556 non-null    float64
dtypes: float64(14), int64(2), object(1)
memory usage: 74.1+ KB


HEI and SCORE_GABAS have 1 missing value. As this source has data with recalls of 24h and +24h, we first look at the specific row before making a desicion

In [18]:
rows_with_na_fg = food_group[food_group.isna().any(axis=1)]
rows_with_na_fg

Code  Codalt      City  r24h  Dairy g  Meats g  Eggs g  Beans g  \
1  1.050010e+14       7  Medellin     2    250.0    294.0  0.0001   0.0001   

   Nuts g  Fruits g  Vegetables g  Cereals g  Tubers g  Fats g  Sugars g  HEI  \
1  0.0001     120.0          52.0      379.0     220.0     9.0     406.0  NaN   

   SCORE_GABAS  
1          NaN

It is an observation with +24h recall -> there is no need to save it, as it will be of no use in the future

#### Step 1.5: otu 

In [19]:
#There are many columns, checking them is slightly different
#Find rows with object dtype
filas_object_otu = otu.columns[otu.dtypes == 'object'].tolist()
filas_object_otu

['samples']

In [20]:
#We double check that the other columns are intt64
filas_int = otu.columns[otu.dtypes == 'int64'].tolist()
print('Int columns', len(filas_int))
print('Non-id columns', otu.shape[1]-1)

Int columns 4738
Non-id columns 4738


In [21]:
#We check for NAs
rows_with_na_otu = otu[otu.isna().any(axis=1)]
rows_with_na_otu

Empty DataFrame
Columns: [samples, Otu00001, Otu00002, Otu00003, Otu00004, Otu00005, Otu00006, Otu00007, Otu00008, Otu00009, Otu00010, Otu00011, Otu00012, Otu00013, Otu00014, Otu00015, Otu00016, Otu00017, Otu00018, Otu00019, Otu00020, Otu00021, Otu00022, Otu00023, Otu00024, Otu00025, Otu00026, Otu00027, Otu00028, Otu00029, Otu00030, Otu00031, Otu00032, Otu00033, Otu00034, Otu00035, Otu00036, Otu00037, Otu00038, Otu00039, Otu00040, Otu00041, Otu00042, Otu00043, Otu00044, Otu00045, Otu00046, Otu00047, Otu00048, Otu00049, Otu00050, Otu00051, Otu00052, Otu00053, Otu00054, Otu00055, Otu00056, Otu00057, Otu00058, Otu00059, Otu00060, Otu00061, Otu00062, Otu00063, Otu00064, Otu00065, Otu00066, Otu00067, Otu00068, Otu00069, Otu00070, Otu00071, Otu00072, Otu00073, Otu00074, Otu00075, Otu00076, Otu00077, Otu00078, Otu00079, Otu00080, Otu00081, Otu00082, Otu00083, Otu00084, Otu00085, Otu00086, Otu00087, Otu00088, Otu00089, Otu00090, Otu00091, Otu00092, Otu00093, Otu00094, Otu00095, Otu00096, Otu00097, Otu00098, Otu00099, ...]
Index: []

[0 rows x 4739 columns]

All values are OK

### Step 2: Create consistency in ID column name 

In [22]:
#Nutrients: Cambiar id por ID
nutrients.rename(columns={nutrients.columns[0]: "ID"}, inplace=True)

#otu: Cambiar samples por ID
otu.rename(columns={otu.columns[0]: "ID"}, inplace=True)

### Step 2.1: Remove rows with duplicate samples

According to existing papers, the following rows are duplicate samples:
- MI_008_H2
- MI_093_H12
- MI_130_H2
- MI_198_H2
- MI_458_H2

So the first thing to do is to remove them

In [23]:
ID_repeticiones = ["MI_008_H2", "MI_093_H12", "MI_130_H2", "MI_198_H2", "MI_458_H2"]

In [24]:
antro_1 = antro_1[~antro_1['ID'].isin(ID_repeticiones)]
antro_1.shape

(441, 32)

### Step 2.2: Consolidate rows for all other sources
It is enough to do isin() with ID. The only exception is food_group,and that will be dealt with later

In [25]:
#antro_2
antro_2 = antro_2[antro_2['ID'].isin(antro_1['ID'])]
antro_2.shape

(441, 6)

In [26]:
#nutrients
nutrients = nutrients[nutrients['ID'].isin(antro_1['ID'])]
nutrients.shape

(441, 30)

In [27]:
#otu
otu = otu[otu['ID'].isin(antro_1['ID'])]
otu.shape

(441, 4739)

### Step 3.1: Consolidate antro
- antro_1 has the base variables
- antro_2 has the socioeconomic variable

In [28]:
#Put socioeconomic in antro_1
antro_complete = pd.merge(antro_1, antro_2[['ID', 'socioeconomic']], on = 'ID', how = 'inner')
antro_complete_columns = antro_complete.columns.tolist()
socio_col = antro_complete_columns.pop()
antro_complete_columns.insert(2, socio_col)
antro_complete = antro_complete[antro_complete_columns]
antro_complete.columns

Index(['ID', 'city', 'socioeconomic', 'sex', 'age', 'age_range', 'HDL', 'LDL',
       'cholesterol', 'triglycerides', 'hsCRP', 'glucose', 'glycosylated_hg',
       'adiponectin', 'insulin', 'HOMA_IR', 'stool_consistency', 'hiden_blood',
       'systolic_bp', 'diastolic_bp', 'bmi', 'bmi_class', 'body_fat', 'waist',
       'medicament', 'fiber', 'per_protein', 'per_animal_protein',
       'per_total_fat', 'per_saturated_fat', 'per_monoinsaturated_fat',
       'per_polyunsaturated_fat', 'per_carbohydrates'],
      dtype='object')

In [29]:
#We can export the file
antro_complete.to_csv('./Consolidated/anthro_data.meta', sep = '\t', index = False)

### Step 3.2: Consolidar datos de food group

In [30]:
#We select the food group rows that are in nutrients
food_group_selected = pd.merge(food_group, nutrients[['ID', 'Codalt']], on = 'Codalt', how = 'inner')
food_group_selected.shape
desired_col_order = ['ID'] + food_group.columns.tolist()
food_group_selected = food_group_selected[desired_col_order]

In [31]:
#Separate food group observations based on recall time
food_group_U24h = food_group_selected[food_group_selected['r24h']<2]
food_group_O24h = food_group_selected[food_group_selected['r24h']>1]

food_group_U24h.shape, food_group_O24h.shape

((441, 18), (94, 18))

### Step 3.3: Export data

In [32]:
#Nutrients
nutrients.to_csv('./Consolidated/nutrients_data.txt', sep = '\t', index = False)

#OTU
otu.to_csv('./Consolidated/otu_data.otus', sep = '\t', index = False)

#Food group under 24h
food_group_U24h = food_group_U24h.sort_values(by = 'Codalt')
food_group_U24h.to_csv('./Consolidated/food_groups_u24h.csv',index = False)

### Step 3.4: Export data without the NA rows

In [33]:
#Antro:
antro_no_NA = antro_complete[~antro_complete['ID'].isin(ids_with_nas)]
antro_complete.to_csv('./ConsolidatednoNA/anthro_data_clean.meta', sep = '\t', index = False)

#Nutrients
nutrients_no_NA = nutrients[~nutrients['ID'].isin(ids_with_nas)]
nutrients_no_NA.to_csv('./ConsolidatednoNA/nutrients_data_clean.txt', sep = '\t', index = False)

#Food group under 24h
food_group_U24h_no_NA = food_group_U24h[~food_group_U24h['ID'].isin(ids_with_nas)]
food_group_U24h_no_NA.to_csv('./ConsolidatednoNA/food_groups_u24h_clean.csv',index = False)

#OTU
otu_no_NA = otu[~otu['ID'].isin(ids_with_nas)]
otu_no_NA.to_csv('./ConsolidatednoNA/otu_data_clean.otus', sep = '\t', index = False)